In [1]:
import librosa
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import urllib.parse
import pymysql

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ------------------------------------ --- 41.0/45.0 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 739.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:

# MySQL database connection details
username = 'root'
pw = 'kaiser123'
host = '127.0.0.1'
database = 'FMA_DB'

# Create a database engine
connection_string = 'mysql+pymysql://root:'+ pw +'@localhost:3306/'
engine = create_engine(connection_string)

# Load and clean CSV files from the FMA DATA SOURCE 
def load_and_clean_csv(file_path):
    df = pd.read_csv(file_path)
    return df

tracks_df = load_and_clean_csv(r'C:\Users\kaise\Downloads\Final Porject Ironhack\FMA metadata\fma_metadata\raw_tracks.csv')
genres_df = load_and_clean_csv(r'C:\Users\kaise\Downloads\Final Porject Ironhack\FMA metadata\fma_metadata\raw_genres.csv')
artists_df = load_and_clean_csv(r'C:\Users\kaise\Downloads\Final Porject Ironhack\FMA metadata\fma_metadata\raw_artists.csv')
albums_df = load_and_clean_csv(r'C:\Users\kaise\Downloads\Final Porject Ironhack\FMA metadata\fma_metadata\raw_albums.csv')
echonest_df = load_and_clean_csv(r'C:\Users\kaise\Downloads\Final_Project_Ironhack\FMA metadata\fma_metadata\echonest_cleaned.csv')


"""# SEND the following tables to the FMA schema in MySQL so as to have a base  line for the EDA and project requirements """
#tracks_df.to_sql(name='track', con=engine, schema='FMA', index=False, if_exists='replace')
#genres_df.to_sql(name='genre', con=engine, schema='FMA', index=False, if_exists='replace')
#artists_df.to_sql(name='artist', con=engine, schema='FMA', index=False, if_exists='replace')
#albums_df.to_sql(name='album', con=engine, schema='FMA', index=False, if_exists='replace')
echonest_df.to_sql(name='echonest',con=engine,schema='FMA',index=False,if_exists='replace')

# Cleaning the data seems to be the idea, in order to have a more concise and clear data base to use when it comes to the analysis 
# this way i can use the data sets in conjunction with others and get rid of redundancies 
# the process is as follows, get rid of redundant collumns that would not really add any information to the data analysis portion in the tables 

14511

In [42]:
# GTZAN database  intergration 
features_df = load_and_clean_csv('/Users/nabilomondi/Downloads/Final Project IRONHACK/GTZAN DATA/Data/features_3_sec.csv')



In [44]:
features_df.shape

(9990, 60)

In [5]:
echonest_df

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,album_date,...,214,215,216,217,218,219,220,221,222,223
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,NaN,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.676960,3.61288,13.316690,262.929749
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,NaN,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.082770,6.01864,16.673548,325.581085
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,NaN,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.926490,5.86635,16.013849,356.755737
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,11/03/2008,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.414010,8.33319,21.317064,483.403809
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,NaN,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.267070,11.20267,26.454180,751.147705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14506,124857,0.007592,0.790364,0.719288,0.853114,0.720715,0.082550,141.332,0.890461,NaN,...,-1.721207,4.686078,0.213789,0.208800,0.007911,0.06395,2.040730,1.97678,8.144532,147.040405
14507,124862,0.041498,0.843077,0.536496,0.865151,0.547949,0.074001,101.975,0.476845,NaN,...,-0.647897,1.282306,0.214586,0.181860,0.011247,0.06240,0.922360,0.85996,1.794739,6.321268
14508,124863,0.000124,0.609686,0.895136,0.846624,0.632903,0.051517,129.996,0.496667,NaN,...,-0.771613,1.623510,0.180471,0.128185,0.010103,0.06222,2.251160,2.18894,5.578341,89.180328
14509,124864,0.327576,0.574426,0.548327,0.452867,0.075928,0.033388,142.009,0.569274,NaN,...,-2.054143,7.927149,0.250178,0.219205,0.014851,0.06390,1.487440,1.42354,2.173092,12.503966


dtype('float64')